In [ ]:
from jax import numpy as jnp
from astrodynx.twobody.ivp import U3
import jax
from jax.typing import ArrayLike
from jax import Array


jax.config.update("jax_enable_x64", True)
jax.devices()

[CudaDevice(id=0), CudaDevice(id=1)]

In [2]:
def U3v2(chi: ArrayLike, alpha: ArrayLike) -> Array:
    alpha_c = jax.lax.complex(alpha, 0.0)
    return jnp.where(
        alpha == 0.0,
        chi**3 / 6.0,
        jax.lax.real(
            (jnp.sqrt(alpha_c) * chi - jnp.sin(jnp.sqrt(alpha_c) * chi))
            / alpha_c
            / jnp.sqrt(alpha_c)
        ),
    )


# --- JIT 编译版本 ---
U3_jit = jax.jit(U3)
U3v2_jit = jax.jit(U3v2)

In [3]:
# --- 测试代码 ---
def run_test():
    # 使用一个固定的随机种子以保证测试的可复现性
    key = jax.random.PRNGKey(0)

    # 生成测试数据
    # chi 可以是任意值
    chi_values = jax.random.uniform(key, shape=(100,), minval=-10.0, maxval=10.0)

    # alpha 的值需要覆盖 >0, <0, 和 =0 三种情况
    key, subkey = jax.random.split(key)
    alpha_pos = jax.random.uniform(subkey, shape=(50,), minval=0.1, maxval=20.0)
    key, subkey = jax.random.split(key)
    alpha_neg = jax.random.uniform(subkey, shape=(49,), minval=-20.0, maxval=-0.1)
    alpha_zero = jnp.array([0.0])
    alpha_values = jnp.concatenate([alpha_pos, alpha_neg, alpha_zero])

    # 为了测试所有 chi 和 alpha 的组合，我们创建网格
    chi_grid, alpha_grid = jnp.meshgrid(chi_values, alpha_values)

    print(f"Testing with chi grid shape: {chi_grid.shape}")
    print(f"Testing with alpha grid shape: {alpha_grid.shape}")

    result1 = U3_jit(chi_grid, alpha_grid)
    result2 = U3v2_jit(chi_grid, alpha_grid)

    # 检查结果是否“足够接近”
    # jnp.allclose 会逐元素比较，看它们的差是否在容差 (tolerance) 以内
    are_they_close = jnp.allclose(result1, result2, atol=1e-6, rtol=1e-6)

    if are_they_close:
        print("\n✅ 测试通过：两个函数的结果在容差范围内一致。")
    else:
        print("\n❌ 测试失败：两个函数的结果不一致。")
        # 找出不一致的地方以便调试
        diff = jnp.abs(result1 - result2)
        max_diff = jnp.max(diff)
        print(f"最大差值: {max_diff}")

        # 找到最大差异发生的位置
        idx = jnp.unravel_index(jnp.argmax(diff), diff.shape)
        print(f"最大差异发生在索引 {idx} 处")
        print(f"  - chi: {chi_grid[idx]}")
        print(f"  - alpha: {alpha_grid[idx]}")
        print(f"  - U3 result: {result1[idx]}")
        print(f"  - U3_ result:   {result2[idx]}")


run_test()

Testing with chi grid shape: (100, 100)
Testing with alpha grid shape: (100, 100)

✅ 测试通过：两个函数的结果在容差范围内一致。


In [4]:
# --- 准备测试数据 ---
key = jax.random.PRNGKey(0)
key_chi, key_alpha = jax.random.split(key)

# 创建一个包含 10 万个元素的大数组
N = 1_000_00
chi = jax.random.uniform(key_chi, (N,)) * jnp.pi * 2

# 案例 1: 所有 alpha > 0
alpha_pos = jax.random.uniform(key_alpha, (N,))

# 案例 2: 所有 alpha < 0
alpha_neg = -jax.random.uniform(key_alpha, (N,))

# 案例 3: alpha 正负混合
alpha_mix = jax.random.normal(key_alpha, (N,))

# 确保 JIT 编译已完成（预热）
U3_jit(chi[:10], alpha_pos[:10]).block_until_ready()
U3v2_jit(chi[:10], alpha_pos[:10]).block_until_ready()

Array([3.24633870e+01, 1.07914181e+00, 6.30361776e+00, 1.30161151e+01,
       8.48328728e-01, 1.55862004e+00, 3.84984327e+00, 1.15542347e+00,
       8.34801777e-03, 4.98232697e+00], dtype=float64)

In [5]:
%timeit U3_jit(chi, alpha_pos).block_until_ready()

%timeit U3v2_jit(chi, alpha_pos).block_until_ready()

86.6 μs ± 7.51 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
The slowest run took 6.69 times longer than the fastest. This could mean that an intermediate result is being cached.
345 μs ± 287 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit U3_jit(chi, alpha_neg).block_until_ready()

%timeit U3v2_jit(chi, alpha_neg).block_until_ready()

89.2 μs ± 6.64 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
199 μs ± 4.47 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%timeit U3_jit(chi, alpha_mix).block_until_ready()

%timeit U3v2_jit(chi, alpha_mix).block_until_ready()

92.9 μs ± 4.3 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
194 μs ± 15.7 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
